In [29]:
import numpy as np

In [30]:
# t 普通數字
# 後面的random是noise
def F1(t):
    return 0.063*(t**3) - 5.284*(t**2) + 4.887*t + 412 + np.random.normal(0, 1)

### b[t] = F1(t) 模擬信號產生的過程。
### 在解 Ax = b

In [10]:
A = np.zeros((1000, 5)) # 每row有5個，有1000row
b = np.zeros((1000,1))

for t in range(0, 1000):
    b[t] = F1(t)
    A[t, 0] = t**4
    A[t, 1] = t**3
    A[t, 2] = t**2
    A[t, 3] = t
    A[t, 4] = 1

# Return the least-squares solution to a linear matrix equation.
x = np.linalg.lstsq(A, b)[0]
# 第一項：Least-squares solution. If b is two-dimensional, the solutions are in the K columns of x.
x

/var/folders/8s/csl8f6rj5lbcl3ch8tybntnw0000gn/T/ipykernel_10592/3573691257.py:13: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x = np.linalg.lstsq(A, b)[0]


array([[ 1.97219007e-12],
       [ 6.29999945e-02],
       [-5.28399539e+00],
       [ 4.88578208e+00],
       [ 4.12077673e+02]])

### 非線性的求解！想要反推已知format：F2 = 0.6t^(1.2) + 100cos(0.4t）
### 下方 b2 即是模擬訊號產生。

In [31]:
def F2(T, A, B, C, D):
    return A*(T**B) + C*np.cos(D*T) + np.random.normal(0, 1, T.shape)

T = np.random.random((1000, 1))*100
# T.shape = (1000, 1)
# Return random floats in the half-open interval [0.0, 1.0)
# 1000 row, 每row一個
b2 = F2(T, 0.6, 1.2, 100, 0.4)

In [33]:
# pop = np.random.randint(0, 2, (N, 40))
# 產生隨機整數 low, high(不包含), size
# pop[i, :][0:10] = array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

# 2**np.arange(10) = array([  1,   2,   4,   8,  16,  32,  64, 128, 256, 512])

# 因為已經知道A, B, C, D的值域範圍，例如：A是從-5.12~5.12，且間格為0.01，共有512種可能，所以用2進位的話，需要10位。
# A, B, C, D各需要10位，所以總共需要40個基因。
# 在篩選時篩基因，但回到係數要回到10進位。
def gene2coef(gene): # 2進位轉10進位
    A = (np.sum(2**np.arange(10)*gene[0:10])-511)/100  # max -5.11~5.12
    B = (np.sum(2**np.arange(10)*gene[10:20])-511)/100
    C = (np.sum(2**np.arange(10)*gene[20:30])-511)
    D = (np.sum(2**np.arange(10)*gene[30:40])-511)/100
    return A,B,C,D

In [13]:
N = 10000   # 每一代人數
G = 10      # 演化次數
survive_rate = 0.05
mutation_rate = 0.001

# round 四捨五入到整數
survive = round(N*survive_rate)
mutation = round(N*40*mutation_rate)

In [14]:
pop = np.random.randint(0, 2, (N, 40))
# 產生隨機的基因當成第一代（low, high(不包含), size）
fit = np.zeros((N, 1))
pop

array([[1, 1, 1, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [34]:
# 繁衍G代
for generation in range(G):
    for i in range(N):
        A, B, C, D = gene2coef(pop[i, :])
        fit[i] = np.mean(abs(F2(T, A, B, C, D) - b2))
    # 真實值跟估計的差異絕對值，想辦法讓他越小越好
    # 存活的人：先排序，選擇前500名。
    # argsort: Returns the indices that would sort an array.
    sortf = np.argsort(fit[:,0])
    pop = pop[sortf, :]

    # 誤差最小的前500個存活，後面用這500個繁衍
    for i in range(survive, N):  #繁衍後501~10000個人
        # 先隨機選取一個爸爸跟一個媽媽，
        fid = np.random.randint(0, survive) # father id
        mid = np.random.randint(0, survive) # mother id
        while(fid == mid):
            mid = np.random.randint(0, survive)
        
        # 先假設兒子完全複製被選到的那一個媽媽的基因
        son = pop[mid, :].copy() # 500筆
        father = pop[fid, :]
        # mask 是“哪個地方”要替代成爸爸的基因
        mask = np.random.randint(0, 2, [1, 40])
        # !!!!!超酷的，用條件來當成index
        son[mask[0, :] == 1] = father[mask[0, :]==1] 
        pop[i, :] = son # 把這個新產生的兒子更新到pop裡面。

# mutation = round(N*40*mutation_rate)
# 繁衍以及突變 一萬個人，有40個新產生的小孩基因突變
    for i in range(mutation):
        # 先選是誰突變
        m = np.random.randint(survive, N)
        # 再選是哪“一些”基因突變了
        n = np.random.randint(0, 40)
        # 遮些突變的要剪1
        pop[m, n] = 1 - pop[m,n]
    # 接著重複產生編號501~10000的小孩。

# 這裡的pop已經是30代以後的了
for i in range(N):
    A, B, C, D = gene2coef(pop[i, :])
    fit[i] = np.mean(abs(F2(T, A, B, C, D)-b2))
sortf = np.argsort(fit[:, 0]) 
# argsort: Returns the indices that would sort an array.
pop = pop[sortf, :]
# pop[0, :] 第31代最棒的基因。
A, B, C, D = gene2coef(pop[0, :])
print(A, B, C, D)

5.1 0.63 5.07 -0.03


#### 關於copy，對於 list 的影響

In [21]:
acc = [[1, 1], [2, 2]]
bcc = acc
# Append
acc.append([3, 3])
# 則bcc也會改變
bcc

[[1, 1], [2, 2], [3, 3]]